In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.graphics.tsaplots import plot_acf

# Read in the data
df = pd.read_csv('cleaned_data.csv')
#df

df.loc[df['id'] == 'AS14.01']


,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,MA
0,2014-03-21,AS14.01,6.200000,0.200000,0.200000,0.134050,17978.907000,6.0,0.0,3139.218,...,49.544,0.000,172.206,239.751,4508.500,915.445,0.000,0.000,598.754,0.134050
1,2014-03-22,AS14.01,6.400000,0.600000,0.400000,0.236880,6142.161000,3.0,1.0,731.429,...,21.076,0.000,0.000,98.143,439.632,37.305,0.000,0.000,117.621,0.190607
2,2014-03-23,AS14.01,6.800000,0.200000,0.800000,0.142741,6773.832001,0.0,0.0,1286.246,...,43.403,0.000,0.000,72.823,900.839,0.000,0.000,30.386,30.086,0.171365
3,2014-03-24,AS14.01,6.000000,0.800000,0.000000,0.078961,15047.351001,10.0,0.0,866.956,...,34.106,0.000,3.010,66.558,3223.626,419.805,0.000,0.000,178.732,0.140922
4,2014-03-25,AS14.01,6.750000,0.500000,0.500000,0.098374,21475.354999,0.0,1.0,1032.768,...,43.054,0.000,0.000,178.819,1919.471,0.000,235.223,0.000,222.893,0.128708
5,2014-03-26,AS14.01,6.600000,-0.200000,0.600000,0.101308,16423.801000,0.0,0.0,1167.497,...,52.331,0.000,0.000,97.498,4592.059,0.000,0.000,0.000,33.365,0.121591
6,2014-03-27,AS14.01,7.000000,0.200000,0.800000,0.159511,17442.149999,2.0,1.0,1229.327,...,42.219,0.000,182.451,58.532,935.381,47.314,0.000,0.000,179.029,0.130728
7,2014-03-28,AS14.01,6.400000,-0.600000,0.600000,0.095698,4923.489000,5.0,0.0,10062.595,...,89.166,233.036,0.000,225.951,512.741,1133.009,0.000,0.000,301.717,0.122759
8,2014-03-29,AS14.01,8.000000,0.200000,1.000000,0.068203,8322.622000,4.0,1.0,1952.630,...,0.000,0.000,0.000,169.594,472.888,52.435,0.000,0.000,600.637,0.110889
9,2014-03-30,AS14.01,7.500000,-0.500000,0.750000,0.049093,4523.214001,0.0,0.0,414.365,...,0.000,0.000,0.000,74.003,167.685,0.000,66.477,0.000,38.296,0.097909


In [111]:
period_length = 5

# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])
distinct_values = df['id'].unique()
new_df = pd.DataFrame()

for value in distinct_values:
    distinct_df = df[df['id'] == value]

    earliest_date = distinct_df['date'].min()
    date_range = pd.date_range(start=earliest_date, freq='D', closed='left', periods=distinct_df.shape[0])
    #print(date_range)

    # Calculate period number
    distinct_df['period'] = (date_range.to_series() - earliest_date).dt.days // period_length + 1
    new_df = new_df.append(distinct_df)

# Reset index
new_df.reset_index(drop=True, inplace=True)
new_df


C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,MA,period
0,2014-03-21,AS14.01,6.20000,0.200000,0.200000,0.134050,17978.907000,6.0,0.0,3139.218,...,0.0,172.206,239.751,4508.500,915.445,0.000,0.000,598.754,0.134050,NaN
1,2014-03-22,AS14.01,6.40000,0.600000,0.400000,0.236880,6142.161000,3.0,1.0,731.429,...,0.0,0.000,98.143,439.632,37.305,0.000,0.000,117.621,0.190607,NaN
2,2014-03-23,AS14.01,6.80000,0.200000,0.800000,0.142741,6773.832001,0.0,0.0,1286.246,...,0.0,0.000,72.823,900.839,0.000,0.000,30.386,30.086,0.171365,NaN
3,2014-03-24,AS14.01,6.00000,0.800000,0.000000,0.078961,15047.351001,10.0,0.0,866.956,...,0.0,3.010,66.558,3223.626,419.805,0.000,0.000,178.732,0.140922,NaN
4,2014-03-25,AS14.01,6.75000,0.500000,0.500000,0.098374,21475.354999,0.0,1.0,1032.768,...,0.0,0.000,178.819,1919.471,0.000,235.223,0.000,222.893,0.128708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,2014-06-05,AS14.24,7.00000,0.000000,0.800000,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.000,47.360,171.964,225.737,0.000,0.000,0.000,0.079928,NaN
1300,2014-06-06,AS14.24,7.50000,0.500000,0.250000,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.000,36.414,0.000,0.000,0.000,0.000,2.072,0.099847,NaN
1301,2014-06-07,AS14.24,8.00000,-1.000000,0.000000,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.000,6.214,106.850,0.000,0.000,0.000,0.000,0.096582,NaN
1302,2014-06-08,AS14.24,6.50000,-1.500000,1.000000,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.000,18.069,190.760,0.000,0.000,0.000,0.000,0.093478,NaN


In [93]:
# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Set 'id' as index
df.set_index('id', inplace=True)

# Find the earliest date for each 'id'
earliest_dates = df.groupby('id')['date'].min()

# Create a dictionary to store the date ranges for each 'id'
date_ranges = {}

# Loop through each 'id' and generate date ranges for each 5-day period
for id, earliest_date in earliest_dates.items():
    id_df = df.loc[id]
    date_range = pd.date_range(start=earliest_date, freq='D', closed='left', periods=(len(id_df) // 5) + 1)
    date_ranges[id] = date_range + pd.Timedelta(days=5)

# Flatten the list of date ranges
all_dates = pd.to_datetime([date for date_range in date_ranges.values() for date in date_range])

# Find the minimum index of 'all_dates' for each 'id'
min_index = df['date'].apply(lambda date: np.argmin(np.abs(all_dates - date)))

# Reset the index of the DataFrame
df.reset_index(inplace=True)

# Define a function to map dates to corresponding period values
def get_period(index):
    return min_index[index] + 1

# Apply the function directly on the index to create a 'period' column
df['period'] = df.index.map(get_period)

# Define custom aggregation functions
agg_dict = {'mood': 'last', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

# Group by 'id' and 'period', and apply the aggregation functions
df = df.groupby(['id', 'period']).agg(agg_dict)

# Reset index
df.reset_index(inplace=True)

# Print the result
df


,id,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,AS14.01,1,6.6,-0.200000,0.600000,0.101308,16423.801000,0.0,0.0,1167.497,...,910.479,52.331,0.000,0.000,97.498,4592.059,0.000,0.000,0.0,33.365
1,AS14.01,2,7.0,0.200000,0.800000,0.159511,17442.149999,2.0,1.0,1229.327,...,639.339,42.219,0.000,182.451,58.532,935.381,47.314,0.000,0.0,179.029
2,AS14.01,3,6.4,-0.600000,0.600000,0.095698,4923.489000,5.0,0.0,10062.595,...,1005.403,89.166,233.036,0.000,225.951,512.741,1133.009,0.000,0.0,301.717
3,AS14.01,4,8.0,0.200000,1.000000,0.068203,8322.622000,4.0,1.0,1952.630,...,1627.429,0.000,0.000,0.000,169.594,472.888,52.435,0.000,0.0,600.637
4,AS14.01,5,7.5,-0.500000,0.750000,0.049093,4523.214001,0.0,0.0,414.365,...,1535.677,0.000,0.000,0.000,74.003,167.685,0.000,66.477,0.0,38.296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,AS14.33,189,6.2,-0.400000,0.400000,0.079744,16450.873999,3.0,1.0,4191.713,...,857.680,0.000,0.000,0.000,139.282,6191.383,144.481,0.000,0.0,97.636
1023,AS14.33,190,7.0,-0.481432,0.369587,0.087960,302107.537994,113.0,22.0,60996.949,...,19293.745,0.000,0.000,488.439,2761.240,129406.795,2506.503,12.089,0.0,1158.387
1024,AS14.33,194,7.0,0.333333,0.333333,0.095028,4944.646001,5.0,1.0,765.403,...,296.785,0.000,0.000,0.000,107.616,4201.894,0.000,27.135,0.0,164.932
1025,AS14.33,195,7.2,-0.600000,1.400000,0.072172,18855.781002,2.0,0.0,2442.832,...,920.187,0.000,0.000,0.000,152.834,6420.129,0.000,0.000,0.0,283.445


In [65]:
# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Set 'id' as index
df.set_index('id', inplace=True)

# Find the earliest date for each 'id'
earliest_dates = df.groupby('id')['date'].min()

# Create a dictionary to store the date ranges for each 'id'
date_ranges = {}

# Loop through each 'id' and generate date ranges for each 5-day period
for id, earliest_date in earliest_dates.items():
    date_range = pd.date_range(start=earliest_date, freq='D', closed='left', periods=(len(df) // 5) + 1)
    date_ranges[id] = date_range + pd.Timedelta(days=5)

# Flatten the list of date ranges
all_dates = pd.to_datetime([date for date_range in date_ranges.values() for date in date_range])

# Map dates to corresponding period values using searchsorted
df['period'] = np.searchsorted(all_dates, df['date']) + 1

# Define custom aggregation functions
agg_dict = {'mood': 'last', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

# Group by 'id' and 'period', and apply the aggregation functions
df = df.groupby(['id', 'period']).agg(agg_dict)

# Reset index
df.reset_index(inplace=True)

# Print the result
df


,id,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,AS14.01,1,6.6,0.350000,0.416667,0.132052,83841.407001,19.0,2.0,8224.114,...,3150.782,243.514,0.000,175.216,753.592,15584.127,1372.555,235.223,30.386,1181.451
1,AS14.01,2,7.0,0.200000,0.800000,0.159511,17442.149999,2.0,1.0,1229.327,...,639.339,42.219,0.000,182.451,58.532,935.381,47.314,0.000,0.000,179.029
2,AS14.01,3,6.4,-0.600000,0.600000,0.095698,4923.489000,5.0,0.0,10062.595,...,1005.403,89.166,233.036,0.000,225.951,512.741,1133.009,0.000,0.000,301.717
3,AS14.01,4,8.0,0.200000,1.000000,0.068203,8322.622000,4.0,1.0,1952.630,...,1627.429,0.000,0.000,0.000,169.594,472.888,52.435,0.000,0.000,600.637
4,AS14.01,5,7.5,-0.500000,0.750000,0.049093,4523.214001,0.0,0.0,414.365,...,1535.677,0.000,0.000,0.000,74.003,167.685,0.000,66.477,0.000,38.296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,AS14.33,6315,6.2,-0.600000,0.400000,0.012704,4089.879001,1.0,2.0,726.391,...,124.694,0.000,0.000,0.000,142.686,2010.364,0.000,0.000,0.000,56.173
1143,AS14.33,6316,8.2,0.000000,1.200000,0.103301,14320.867998,10.0,1.0,2551.046,...,614.706,0.000,0.000,357.909,529.946,5361.211,0.000,0.000,0.000,30.666
1144,AS14.33,6317,7.0,-1.333333,1.000000,0.169354,3569.341000,5.0,1.0,400.034,...,136.738,0.000,0.000,0.000,29.202,1789.922,0.939,0.000,0.000,3.199
1145,AS14.33,6318,6.8,-0.800000,-0.400000,0.192901,9497.646999,4.0,0.0,3044.030,...,260.220,0.000,0.000,0.000,52.610,3166.409,1052.648,8.072,0.000,232.825


In [101]:
df.loc[df['id'] == 'AS14.33']

,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,MA
1060,2014-04-16,AS14.33,7.000000,0.333333,0.333333,0.095028,4944.646001,5.0,1.0,765.403,...,0.000,0.0,0.000,107.616,4201.894,0.000,27.135,0.0,164.932,0.095028
1061,2014-04-17,AS14.33,7.200000,-0.600000,1.400000,0.072172,18855.781002,2.0,0.0,2442.832,...,0.000,0.0,0.000,152.834,6420.129,0.000,0.000,0.0,283.445,0.082457
1062,2014-04-18,AS14.33,7.400000,-0.800000,0.800000,0.036852,11030.036000,3.0,1.0,1555.364,...,0.000,0.0,0.000,70.610,6281.793,0.000,0.000,0.0,102.369,0.064124
1063,2014-04-19,AS14.33,7.666667,0.000000,1.000000,0.259306,10002.624000,2.0,0.0,2879.461,...,0.000,0.0,0.000,55.455,3960.064,0.000,0.000,0.0,83.939,0.128428
1064,2014-04-20,AS14.33,7.400000,0.000000,1.200000,0.030758,10896.482999,0.0,0.0,669.192,...,0.000,0.0,0.000,60.848,7490.158,0.000,0.000,0.0,3.010,0.100389
1065,2014-04-21,AS14.33,7.200000,0.000000,0.600000,0.039461,10135.543000,0.0,0.0,621.054,...,0.000,0.0,0.000,63.275,9143.261,3.019,0.000,0.0,4.208,0.084564
1066,2014-04-22,AS14.33,6.200000,-0.600000,-0.200000,0.085497,20483.757999,7.0,0.0,3783.640,...,0.000,0.0,0.000,78.571,7711.711,52.436,0.000,0.0,90.132,0.084789
1067,2014-04-23,AS14.33,5.200000,-0.200000,-0.600000,0.007715,6824.430000,1.0,0.0,865.250,...,0.000,0.0,0.000,68.263,4779.349,0.000,0.000,0.0,0.000,0.067254
1068,2014-04-24,AS14.33,7.000000,-0.250000,1.000000,0.134273,8703.779000,5.0,3.0,3016.506,...,0.000,0.0,0.000,67.548,3687.524,50.712,0.000,0.0,90.989,0.081835
1069,2014-04-25,AS14.33,8.400000,0.800000,0.800000,0.146450,9947.085998,31.0,2.0,4057.256,...,0.000,0.0,0.000,52.192,2758.260,36.169,0.000,0.0,0.000,0.095408


In [55]:
# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Set 'id' as index
df.set_index('id', inplace=True)

# Find the earliest date for each 'id'
earliest_dates = df.groupby('id')['date'].min()

# Create a dictionary to store the date ranges for each 'id'
date_ranges = {}

# Loop through each 'id' and generate date ranges for each 5-day period
for id, earliest_date in earliest_dates.items():
    date_range = pd.date_range(start=earliest_date, freq='5D', closed='left', periods=(len(df) // 5) + 1)
    date_ranges[id] = date_range

# Flatten the list of date ranges
all_dates = pd.to_datetime([date for date_range in date_ranges.values() for date in date_range])

# Map dates to corresponding period values using searchsorted
df['period'] = np.searchsorted(all_dates, df['date']) + 1

# Define custom aggregation functions
agg_dict = {'mood': 'last', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

# Group by 'id' and 'period', and apply the aggregation functions
df = df.groupby(['id', 'period']).agg(agg_dict)

# Reset index
df.reset_index(inplace=True)

# Print the result
df


,id,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,AS14.01,1,6.20,0.200000,0.200000,0.134050,17978.907000,6.0,0.0,3139.218,...,1007.456,49.544,0.000,172.206,239.751,4508.500,915.445,0.000,0.000,598.754
1,AS14.01,2,6.60,0.380000,0.460000,0.131653,65862.500001,13.0,2.0,5084.896,...,2143.326,193.970,0.000,3.010,513.841,11075.627,457.110,235.223,30.386,582.697
2,AS14.01,3,7.40,-0.140000,0.750000,0.088602,47048.309000,17.0,2.0,16620.322,...,6566.768,172.640,284.212,182.451,796.634,3439.350,1416.139,95.767,0.000,1329.984
3,AS14.01,4,6.80,-0.190000,0.300000,0.103058,75654.094003,11.0,1.0,24438.349,...,5289.240,871.837,234.741,182.869,1134.567,14072.505,1979.889,0.000,0.000,1289.427
4,AS14.01,5,7.40,-0.570000,0.510000,0.077951,55269.223002,17.0,0.0,8722.700,...,6900.477,65.259,115.465,40.302,953.555,8453.066,60.247,0.000,0.000,1243.249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,AS14.33,12,6.00,-0.116667,0.313333,0.060332,59691.757998,9.0,2.0,7511.488,...,3898.637,0.000,0.000,0.000,566.346,22361.588,237.870,2.007,0.000,64.356
262,AS14.33,13,6.25,-0.130000,0.390000,0.022743,46369.721001,13.0,1.0,6711.250,...,2839.927,0.000,0.000,0.000,314.390,24230.355,0.000,2.010,0.000,40.459
263,AS14.33,14,6.25,-0.470000,0.106667,0.045012,39257.282000,9.0,2.0,6027.681,...,3691.500,0.000,0.000,0.000,345.870,23037.320,0.000,0.000,0.000,105.820
264,AS14.33,15,6.80,-0.626667,0.440000,0.118693,50402.922997,30.0,7.0,7770.240,...,1825.127,0.000,0.000,488.439,990.281,17769.170,1053.587,8.072,0.000,322.863


In [102]:
# df.loc['AS14.01']
# result.loc['AS14.01']
df.loc[df['id'] == 'AS14.33']

,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,MA
1060,2014-04-16,AS14.33,7.000000,0.333333,0.333333,0.095028,4944.646001,5.0,1.0,765.403,...,0.000,0.0,0.000,107.616,4201.894,0.000,27.135,0.0,164.932,0.095028
1061,2014-04-17,AS14.33,7.200000,-0.600000,1.400000,0.072172,18855.781002,2.0,0.0,2442.832,...,0.000,0.0,0.000,152.834,6420.129,0.000,0.000,0.0,283.445,0.082457
1062,2014-04-18,AS14.33,7.400000,-0.800000,0.800000,0.036852,11030.036000,3.0,1.0,1555.364,...,0.000,0.0,0.000,70.610,6281.793,0.000,0.000,0.0,102.369,0.064124
1063,2014-04-19,AS14.33,7.666667,0.000000,1.000000,0.259306,10002.624000,2.0,0.0,2879.461,...,0.000,0.0,0.000,55.455,3960.064,0.000,0.000,0.0,83.939,0.128428
1064,2014-04-20,AS14.33,7.400000,0.000000,1.200000,0.030758,10896.482999,0.0,0.0,669.192,...,0.000,0.0,0.000,60.848,7490.158,0.000,0.000,0.0,3.010,0.100389
1065,2014-04-21,AS14.33,7.200000,0.000000,0.600000,0.039461,10135.543000,0.0,0.0,621.054,...,0.000,0.0,0.000,63.275,9143.261,3.019,0.000,0.0,4.208,0.084564
1066,2014-04-22,AS14.33,6.200000,-0.600000,-0.200000,0.085497,20483.757999,7.0,0.0,3783.640,...,0.000,0.0,0.000,78.571,7711.711,52.436,0.000,0.0,90.132,0.084789
1067,2014-04-23,AS14.33,5.200000,-0.200000,-0.600000,0.007715,6824.430000,1.0,0.0,865.250,...,0.000,0.0,0.000,68.263,4779.349,0.000,0.000,0.0,0.000,0.067254
1068,2014-04-24,AS14.33,7.000000,-0.250000,1.000000,0.134273,8703.779000,5.0,3.0,3016.506,...,0.000,0.0,0.000,67.548,3687.524,50.712,0.000,0.0,90.989,0.081835
1069,2014-04-25,AS14.33,8.400000,0.800000,0.800000,0.146450,9947.085998,31.0,2.0,4057.256,...,0.000,0.0,0.000,52.192,2758.260,36.169,0.000,0.0,0.000,0.095408


In [61]:
# Still need to remove the first column
df.to_csv('cleaned_data_time.csv', index=False)

In [60]:
result.loc[result['id'] == 'AS14.01']

,id,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,AS14.01,1,6.200000,0.200000,0.200000,0.134050,17978.907000,6.0,0.0,3139.218,...,1007.456,49.544,0.000,172.206,239.751,4508.500,915.445,0.000,0.000,598.754
1,AS14.01,2,6.600000,0.380000,0.460000,0.131653,65862.500001,13.0,2.0,5084.896,...,2143.326,193.970,0.000,3.010,513.841,11075.627,457.110,235.223,30.386,582.697
2,AS14.01,3,7.400000,-0.140000,0.750000,0.088602,47048.309000,17.0,2.0,16620.322,...,6566.768,172.640,284.212,182.451,796.634,3439.350,1416.139,95.767,0.000,1329.984
3,AS14.01,4,6.800000,-0.190000,0.300000,0.103058,75654.094003,11.0,1.0,24438.349,...,5289.240,871.837,234.741,182.869,1134.567,14072.505,1979.889,0.000,0.000,1289.427
4,AS14.01,5,7.400000,-0.570000,0.510000,0.077951,55269.223002,17.0,0.0,8722.700,...,6900.477,65.259,115.465,40.302,953.555,8453.066,60.247,0.000,0.000,1243.249
5,AS14.01,6,6.400000,-0.880000,0.790000,0.068051,55854.076998,9.0,2.0,6676.829,...,11622.090,278.872,0.000,0.000,480.205,8245.936,76.732,0.000,0.000,1832.847
6,AS14.01,7,7.250000,-0.230000,0.700000,0.088262,62668.450000,6.0,3.0,5179.988,...,2540.095,225.398,29.084,0.000,208.020,8648.334,192.764,0.000,0.000,788.816
7,AS14.01,8,7.200000,-0.080000,0.960000,0.079984,69919.020998,8.0,1.0,9069.912,...,6639.131,320.778,0.000,0.000,264.387,7294.348,182.778,0.000,0.000,1945.212
8,AS14.01,9,7.800000,-0.320000,0.850000,0.103643,49039.307997,6.0,4.0,9602.710,...,5251.610,867.542,333.305,0.000,1401.728,5320.431,785.553,3.148,0.000,1279.025
9,AS14.01,10,7.709987,-0.323551,0.938835,0.066036,46469.435999,7.0,5.0,5621.591,...,4832.624,2687.165,0.000,28.098,152.967,6415.591,143.631,30.102,0.000,670.588
